In [1]:
from flask import render_template, request
from app import app
from app import db
from models import Historical, Forecast, Station, Current
import datetime

import plotly
from plotly import graph_objs as go
from plotly import plotly as py 
from plotly import tools
import matplotlib as mpl

import pandas as pd
import numpy as np 
import json

In [2]:
with app.test_request_context(
        '/', data={'timeframe': 'Days'}):
    print(request.values['timeframe'])

Days


In [10]:
hoursBack=12
hoursForward=144
    
now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)
query = Forecast.query.filter(
    Forecast.id >= now - datetime.timedelta(hours=hoursBack),
    Forecast.id <= now + datetime.timedelta(hours=hoursForward),
    Forecast.retrieval_time == db.session.query(func.max(Forecast.retrieval_time))[0]            
).order_by(Forecast.id)



In [4]:
from sqlalchemy.sql.expression import func

In [11]:
query

In [ ]:
func.max(Forecast.retrieval_time)

In [ ]:
def create_future(hoursBack, linewidth=4, skip=False, daily=False, futureOnly=False):
    now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)
    
    if futureOnly==True:
    	data = []
    	start = now
    else:
    	data=[create_actuals(linewidth=linewidth, hoursBack=hoursBack, showlegend=False), create_now()]
    	start = now - datetime.timedelta(hours=hoursBack)

    # option to query every other hour for better performance
    if skip==True:
    	hoursList = range(1,hoursBack)[::2]
    elif daily==True:
    	hoursList = range(1, hoursBack)[::6]
    else:
    	hoursList = range(1,hoursBack)

    for n in hoursList:
        query = Forecast.query.filter(
            Forecast.retrieval_time == now - datetime.timedelta(hours=n)
        ).order_by(Forecast.id)

        x = [x.id for x in query if x.id >= start]
        y = [x.drybulb for x in query if x.id >= start]

        # Colour gradient
        c1='#FF0000' #more distant
        c2= '#000099'
        mix=1-n/hoursBack

        if n == hoursList[0]:
        	hover = 'x+y+name'
        else:
        	hover = 'none'

        forecast = go.Scatter(
                x=x,
                y=y,
                name='forecast {} h ago '.format(n),
                hoverinfo = hover,
                line=dict(
                    color = (colorFader(c1,c2,mix)),
                    width = linewidth*(1/n)**(5/8),
                    ),
                mode='lines',
                showlegend=False
                )

        data.append(forecast)

    
    fig = go.Figure(data=data, layout=create_layout())
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON

In [ ]:
hoursBack = 145
hoursList = range(1, hoursBack)[::2]
for i in hoursList:
    print(i)

In [ ]:
def index():
	try:
		timeframe = request.args.get('timeframe', 0, type=str)
		if timeframe == 'Hours':
			return render_template(
    				'index.html', 
    				timeframes=timeframes, 
    				plot=create_plot1()
    				)

		elif timeframe == 'Days':
			return render_template(
    				'index.html', 
    				timeframes=timeframes, 
    				plot=create_plot2()
    				)
		else:
			return render_template(
    				'index.html', 
    				timeframes=timeframes, 
    				plot=create_plot2()
    				)

	except Exception as e:
		return str(e)



In [ ]:
create_plot()

In [ ]:
create_plot2()

In [ ]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

def create_plot():
    # Variables
    w=4
    h=24
    now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)
    data=[]

    # Layout
    layout = go.Layout(
    title=go.layout.Title(
        text='Weather Forecast Tracker',
            font=dict(
                family='Helvetica, monospace',
                size=20
	            )
    ),

    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Week of {}'.format((now - datetime.timedelta(days=now.weekday())).date()),
            font=dict(
                family='Helvetica, monospace',
                size=18
            )
        )
    ),

    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Temperature \u2103',
            font=dict(
                family='Helvetica, monospace',
                size=18
	            )
	        )
	    )
	)

    # actuals
    x0 = [x.id for x in Current.query.filter(
    	Current.id >= now - datetime.timedelta(days=7)
    	).order_by(Current.id)]
    y0 = [x.drybulb for x in Current.query.filter(
    	Current.id >= now - datetime.timedelta(days=7)
    	).order_by(Current.id)]



    actuals = go.Scatter(
        x=x0,
        y=y0,
        name='actual temperature',
        mode='lines',
        line=dict(
            color = ('#FF007F'),
            width = w)
        )   

    data.append(actuals)

    # hourly for past n hours 
    for n in range(1,h):
        x = [x.id for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(minutes=60*n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=60*n-10)
        ).order_by(Forecast.id)]

        y = [x.drybulb for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(minutes=60*n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=60*n-10)
        ).order_by(Forecast.id)]

        # Colour gradient
        c1='#3399FF' #more distant
        c2='#990099' #nearer
        mix=1-n/h

        if n in (1, h-1):
        	l= True
        else: 
        	l= False

        forecasts = go.Scatter(
                x=x,
                y=y,
                name='forecast {} h ago'.format(n),
                line=dict(
                    color = (colorFader(c1,c2,mix)),
                    width = w*(1/n)**(5/8),
                    ),
                showlegend=l,
                )
        data.append(forecasts)



    data=data[::-1]
    
    fig = go.Figure(data=data, layout=layout)
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON

def create_plot2():
    
    now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)
    
    def generate_relative_sets(q=12):
        data=[]
        w=4

        for m in range (1,q):  

            # Colour gradient
            c1='#3399FF' #more distant
            c2='#990099' #nearer
            mix=1-m/q

            relative_forecasts = go.Scatter(
                x=relative_set(m)[0],
                y=relative_set(m)[1],
                name='forecast {} h from time'.format(m),
                line=dict(
                    color = (colorFader(c1,c2,mix)),
                    width = w*(1/m)**(5/8),
                    ),
                #showlegend=l,
                )
            data.append(relative_forecasts)
        
        return data

    def relative_set(m):
        h = 48
        x = [x.id for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(hours=h),
            Forecast.retrieval_time == Forecast.id - datetime.timedelta(hours=m)            
        ).order_by(Forecast.id)]

        y = [x.drybulb for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(hours=h),
            Forecast.retrieval_time == Forecast.id - datetime.timedelta(hours=m)
        ).order_by(Forecast.id)]

        return x,y

    # Layout
    layout = go.Layout(
    title=go.layout.Title(
        text='Weather Forecast Tracker 2',
        font=dict(
                family='Helvetica, monospace',
                size=20
                )
    ),

    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Week of {}'.format((now - datetime.timedelta(days=now.weekday())).date()),
            font=dict(
                family='Helvetica, monospace',
                size=18
            )
        )
    ),

    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Temperature \u2103',
            font=dict(
                family='Helvetica, monospace',
                size=18
                )
            )
        )
    )

    data = generate_relative_sets()
    fig = go.Figure(data=data, layout=layout)
    graphJSON2 = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON2


In [ ]:
    def generate_relative_sets(q):
        data=[]
        w=4

        for m in range (1,q):  

            # Colour gradient
            c1='#3399FF' #more distant
            c2='#990099' #nearer
            mix=1-m/12

            relative_forecasts = go.Scatter(
                x=relative_set(m)[0],
                y=relative_set(m)[1],
                name='forecast {} h from time'.format(m),
                line=dict(
                    color = (colorFader(c1,c2,mix)),
                    width = w*(1/m)**(5/8),
                    ),
                #showlegend=l,
                )
            data.append(relative_forecasts)
        
        return data

In [ ]:
    def relative_set(m):
        h = 48
        x = [x.id for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(hours=h),
            Forecast.retrieval_time == Forecast.id - datetime.timedelta(hours=m)            
        ).order_by(Forecast.id)]

        y = [x.drybulb for x in Forecast.query.filter(
            Forecast.retrieval_time >= now - datetime.timedelta(hours=h),
            Forecast.retrieval_time == Forecast.id - datetime.timedelta(hours=m)
        ).order_by(Forecast.id)]

        return x,y

In [ ]:
create_plot2()

In [ ]:
now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)


In [ ]:
data[4]

In [ ]:
generate_relative_sets()

In [ ]:
relative_set(5)[0]

In [ ]:
def generate_relative_sets():
    for m in range (1,48):   
        relative_forecasts = go.Scatter(
            x=relative_set(m)[0],
            y=relative_set(m)[1],
            name='forecast {} h from time'.format(m),
            line=dict(
                #color = (colorFader(c1,c2,mix)),
                #width = w*(1/n)**(5/8),
                ),
            #showlegend=l,
            )
        data.append(relative_forecasts)

In [ ]:
def relative_set(m):
    
    x=[]
    y=[]
    for n in range (1,48):

        x.extend(Forecast.query.filter(
        Forecast.id == now - datetime.timedelta(hours=n),
        Forecast.retrieval_time == Forecast.id - 
            datetime.timedelta(hours=m)
        ).with_entities('id'))
        
        y.extend(Forecast.query.filter(
        Forecast.id == now - datetime.timedelta(hours=n),
        Forecast.retrieval_time == Forecast.id - 
            datetime.timedelta(hours=m)
        ).with_entities('drybulb'))
        
    return x,y
       
        

In [ ]:
relative_set(6)[0]

In [ ]:
x = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=n),
    Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]


In [ ]:
Forecast.query.filter(
        Forecast.id == now - datetime.timedelta(hours=24),
        Forecast.retrieval_time == Forecast.id - datetime.timedelta
    (hours=6)
        ).all('drybulb')



In [ ]:
Forecast.retrieval_time == now - datetime.timedelta(hours=m),



In [ ]:
hourly_plot()
data

In [ ]:
data = []
def hourly_plot():

    for n in range(24):
        x = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

        y = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]   


        forecasts = go.Scatter(
                x=x,
                y=y,
                name='forecasts_{}h'.format(n),
                line=dict(
                    color = ('rgb(0,76,153)'),
                    width = 2,)
                )
        data.append(forecasts)


In [ ]:
data = [historicals, actuals, forecasts_latest, forecasts_6h, forecasts_1d, forecasts_2d]
fig = go.Figure(data=data, layout=layout)
graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
return graphJSON



In [ ]:
    # 6h   
    x4 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=4),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

    y4 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=4),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]


In [ ]:
@app.route('/')
def index():   

    plot = create_plot()
    return render_template("index.html", plot=plot)    
    
def create_plot():
    N = 40
    now = datetime.datetime.now()
    x00 = [x.id for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]
    y00 = [x.drybulb for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]

    # actuals
    x0 = [x.id for x in Current.query.filter(Current.id >= now - datetime.timedelta(days=7)).order_by(Current.id)]
    y0 = [x.drybulb for x in Current.query.filter(Current.id >= now - datetime.timedelta(days=7)).order_by(Current.id)]

    # latest forecast (retrieved within the last hour)   
    x1 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=1))
          .order_by(Forecast.id)]

    y1 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=1))
          .order_by(Forecast.id)]

    # 6h   
    x4 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=6),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

    y4 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=6),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]
        
    # retrieved on roughly this hour, 1d ago
    # datetime greater than now minus 1d; less than five minutes after now minus 1d   
    x2 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    y2 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    # retrieved on roughly this hour, 2d ago
    # datetime greater than now minus 2d; less than five minutes after now minus 2d   
    x3 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    y3 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    historicals = go.Scatter(
            x=x00,
            y=y00,
            name='historicals',
            line=dict(
                color = ('rgb(192,192,192)'),
                width = 2,)
            )

    actuals = go.Scatter(
            x=x0,
            y=y0,
            name='actuals',
            mode='lines',
            line=dict(
                color = ('rgb(255,0,127)'),
                width = 3,)
            )
    
    forecasts_latest = go.Scatter(
            x=x1,
            y=y1,
            name='forecasts_latest',
            line=dict(
                color = ('rgb(0,51,102)'),
                width = 2,)
            )

    forecasts_1d = go.Scatter(
            x=x2,
            y=y2,
            name='forecasts_1d',
            line=dict(
                color = ('rgb(0,76,153)'),
                width = 2,)
            )

    forecasts_2d = go.Scatter(
            x=x3,
            y=y3,
            name='forecasts_2d',
            line=dict(
                color = ('rgb(0,102,204)'),
                width = 2,)
            )

    forecasts_6h = go.Scatter(
            x=x4,
            y=y4,
            name='forecasts_6h',
            line=dict(
                color = ('rgb(0,128,255)'),
                width = 2,)
            )

    layout = go.Layout(
    title=go.layout.Title(
        text='Weather Forecast Tracker',
            font=dict(
                family='Helvetica, monospace',
                size=20
	            )
    ),

    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Week of {}'.format((now - datetime.timedelta(days=now.weekday())).date()),
            font=dict(
                family='Helvetica, monospace',
                size=18
            )
        )
    ),

    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Degrees C',
            font=dict(
                family='Helvetica, monospace',
                size=18
	            )
	        )
	    )
	)

    

    data = [historicals, actuals, forecasts_latest, forecasts_6h, forecasts_1d, forecasts_2d]
    fig = go.Figure(data=data, layout=layout)
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON


In [ ]:
create_plot()